In [ ]:
!pip install --upgrade pip

In [2]:
!pip install geopandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 22.8 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 28.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 19.6 MB/s eta 0:00:00:00:01


In [4]:
import os, sys
from time import time
from glob import glob
import datetime as dt
import numpy as np

import pandas as pd
import matplotlib.pyplot as plt

from IPython.display import clear_output

# import cupy as cp

import geopandas as gpd
#from osgeo import osr, gdal
import netCDF4 as nc

ModuleNotFoundError: No module named 'netCDF4'

In [2]:
import boto3
from botocore.exceptions import NoCredentialsError

# set up s3 location
s3 = boto3.client('s3', region_name='us-east-1')  # Replace 'us-west-2' with your AWS region

# some tests. keeping them for demo. skip when running jobs.

In [3]:
test_uri = 's3://pepper-dataset/crop_data/crop_IOWA/Iowa_corn_2019.csv'
df = pd.read_csv(test_uri)
print(df.shape)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/fsspec/registry.py:275: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


(425, 21)


In [24]:
!ls /vsis3/pepper-dataset/county_mask_data/mask_2/  

ls: cannot access /vsis3/pepper-dataset/county_mask_data/mask_2/: No such file or directory


In [23]:
gdal.Open('/vsis3/pepper-dataset/county_mask_data/mask_2/Illinois_Carroll.tif')

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x7fe86d554ea0> >

In [32]:
with nc.Dataset('/vsis3/pepper-dataset/era5-unzipped/2020/2020-06_part1.nc', 'r') as ds:
    print(ds.variables.keys())

FileNotFoundError: [Errno 2] No such file or directory: '/vsis3/pepper-dataset/era5-unzipped/2020/2020-06_part1.nc'

In [33]:
with nc.Dataset('s3://pepper-dataset/era5-unzipped/2020/2020-06_part1.nc', 'r') as ds:
    print(ds.variables.keys())

syntax error, unexpected WORD_WORD, expecting SCAN_ATTR or SCAN_DATASET or SCAN_ERROR
context: <?xml^ version="1.0" encoding="UTF-8"?><Error><Code>AccessDenied</Code><Message>Access Denied</Message><RequestId>A8PP052W55MZSTKX</RequestId><HostId>+BQeBhBmOMvLIQMvcHAAKWwAIVzwswTFeQC6Iye3qgPZGfUgC527kRQQkE8doLA/2Ahgp2QycjM=</HostId></Error>


OSError: [Errno -78] NetCDF: Authorization failure: 's3://pepper-dataset/era5-unzipped/2020/2020-06_part1.nc'

In [3]:
gds = gdal.Open('/vsis3/pepper-dataset/era5-unzipped/2020/2020-06_part1.nc')

NameError: name 'gdal' is not defined

In [41]:
era_subsets = gds.GetSubDatasets()
print(era_subsets[:2])

[('NETCDF:"/vsis3/pepper-dataset/era5-unzipped/2020/2020-06_part1.nc":t2m', '[720x251x601] t2m (16-bit integer)'), ('NETCDF:"/vsis3/pepper-dataset/era5-unzipped/2020/2020-06_part1.nc":rsn', '[720x251x601] rsn (16-bit integer)')]


In [42]:
gds1 = gdal.Open(era_subsets[0][0])

In [45]:
testarr = gds1.ReadAsArray()

In [50]:
desc = gds1.GetDescription()
print(desc)
print(desc.split(':')[-1])

NETCDF:"/vsis3/pepper-dataset/era5-unzipped/2020/2020-06_part1.nc":t2m
t2m


In [67]:
gds1_meta = gds1.GetMetadata()
gds1_meta.keys()
time_set = eval(gds1_meta['NETCDF_DIM_time_VALUES'])


In [68]:
type(time_set)

set

In [70]:
time_list = []
for t1 in time_set:
    t2 = dt.datetime(1900,1,1) + dt.timedelta(hours = int(t1))
    time_list.append(t2)

In [46]:
testarr.shape

(720, 251, 601)

# prepare masks

In [6]:
flist_raw = !aws s3 ls s3://pepper-dataset/county_mask_data/mask_2/

In [25]:
# 所有mask文件
pad = '2024-06-16 02:14:19      23897 '
len(pad)
mask_list = []
for f in flist_raw:
    if "tif" in f:
        fname = f[31:]
        fpath = f"/vsis3/pepper-dataset/county_mask_data/mask_2/{fname}"
        mask_list.append(fpath)
mask_list.sort()
print(len(mask_list))

141


In [26]:
# 遍历所有 mask 文件，获取某个county的这个变量的数据
mask_dict = {}
for mask_file in mask_list:
    fname = os.path.basename(mask_file)
    location = fname.split('.')[0]
    mask_dict[location] = {}
    # read mask tiff
    gd = gdal.Open(mask_file)
    garr = gd.GetRasterBand(1).ReadAsArray()

    mask_dict[location] = garr

# Do masking with gdal

In [28]:
flist_raw = !aws s3 ls s3://pepper-dataset/era5-unzipped/2020/
flist = []
for f in flist_raw:
    f2 = f.split(' ')[-1]
    f3 = f'/vsis3/pepper-dataset/era5-unzipped/{f2}'
    flist.append(f3)

In [7]:
geotrans = [-125.05, 0.1, 0, 50.05, 0, -0.1]
vlist2 = ['evabs', 'evavt', 'sp', 'sshf', 'ssrd', 'strd', 'tp']
vlist1 = ['t2m', 'rsn', 'sde', 'stl1', 'stl2', 'stl3', 'stl4', 'tsn', 'swvl1', 'swvl2', 'swvl3', 'swvl4']

for year in range(2020, 2021):
    # 获取某一年的文件
    flist_raw = !aws s3 ls s3://pepper-dataset/era5-unzipped/2020/
    flist = []
    for f in flist_raw:
        f2 = f.split(' ')[-1]
        f3 = f'/vsis3/pepper-dataset/era5-unzipped/{f2}'
        flist.append(f3)
    flist.sort()

    # 遍历该年的所有netcdf文件， 确定是part1还是part2
    for ncf in flist:
        ncfname = os.path.basename(ncf)
        yearmonth = ncfname.split('_')[0]
        month_int = int(yearmonth.split('-')[1])

        # 跳过已经完成的部分
        # if year == 2020:
        #     if month_int not in [9, 10]:
        #         continue
        # 所有county数据保存在 2 个大表里
        # 每个大表都有一个 time 列表，然后每个 county 单独一个 dict
        outd = {'time': []}
        # read basic variables
        with nc.Dataset(ncf,'r') as ds:
            var_all = list(ds.variables.keys())
            file_type = 0
            if ncf.split('_')[-1] == 'part1.nc':
                file_type = 1
            elif ncf.split('_')[-1] == 'part2.nc':
                file_type = 2
            else:
                raise ValueError('Unknown file type')

        # set variable list
        if file_type == 1:
            vlist = vlist1
            # continue
        elif file_type == 2:
            vlist = vlist2

        # 打开netcdf文件，先获取时间信息
        with nc.Dataset(ncf,'r') as ds:
            timevar = ds.variables['time'][:]
            timelist = [
                dt.datetime(1900, 1, 1, 0, 0) + dt.timedelta(hours = int(x)) for x in timevar
                ]

            # 根据 file_type 选择添加数据到 all_dict1 还是 all_dict2
            if file_type == 1:
                vlist = vlist1
            else:
                vlist = vlist2
            outd['time'].extend(timelist)
            # 遍历所有变量名，获取全美的数据
            for v in vlist:
                with open(f'/content/drive/MyDrive/2024/ai_pepper/era5/mask2_log_YuhuaSitu2.log','a') as fp:
                    now_str = dt.datetime.strftime(dt.datetime.now(), '%Y%m%dT%H%M%S')
                    fp.writelines(f'{ncf} - {v} - {now_str}\n')
                arr = ds.variables[v][:]   # [time, lat, lon]
                # arr2 = cp.asarray(arr1.data)
                # arr = cp.where(arr2 < -30000, cp.nan, arr2)

                for location in mask_dict.keys():
                    print(ncf)
                    print(v)
                    print(location)
                    if location not in outd.keys():
                        outd[location] = {}
                    if v not in outd[location].keys():
                        outd[location][v] = np.array([])
                    garr1 = mask_dict[location]
                    garr_inds = np.where(garr1 > -10)
                    # arr_masked = garr1 * arr
                    arr_masked = arr[:, garr_inds[0], garr_inds[1]]
                    arr_out = np.mean(arr_masked, axis=1)
                    # arr_out = cp.nanmean(arr_masked, axis=(1,2))
                    # print('einsum')
                    # arr_out = np.einsum("ijk,jk->ijk", arr, garr)
                    # arr_out = np.einsum("ijk->i", arr_masked)
                    print('saving to outd (dictionary)')
                    outd[location][v] = np.concatenate(
                        [outd[location][v], arr_out],
                        axis=0
                        )
                    clear_output(wait=True)


        # 每个月保存一次
        partname = ncfname.split('_')[1].split('.')[0]
        errlist = []
        for location in outd.keys():
            if location == 'time':
                continue
            os.makedirs(f'/content/drive/MyDrive/2024/ai_pepper/era5/county_env2/{location}', exist_ok=True)
            try:
                outd1 = {}
                for key in outd[location].keys():
                    outd1[key] = outd[location][key]# .get()
                t2 = pd.DataFrame(outd1)
                t2['datetime'] = outd['time']
                t2.to_csv(f'/content/drive/MyDrive/2024/ai_pepper/era5/county_env2/{location}/{location}_{yearmonth}_{partname}.csv', index=False)
            except:
                print(f'error with location: {location}')
                errlist.append(location)


/content/drive/MyDrive/2024/ai_pepper/era5/unzip/2020/2020-10_part2.nc
tp
Iowa_Wright
saving to outd (dictionary)


# do masking with netcdf4

In [ ]:
flist_raw = !aws s3 ls s3://pepper-dataset/era5-unzipped/2020/
flist = []
for f in flist_raw:
    f2 = f.split(' ')[-1]
    f3 = f'/vsis3/pepper-dataset/era5-unzipped/{f2}'
    flist.append(f3)

In [ ]:
geotrans = [-125.05, 0.1, 0, 50.05, 0, -0.1]
vlist2 = ['evabs', 'evavt', 'sp', 'sshf', 'ssrd', 'strd', 'tp']
vlist1 = ['t2m', 'rsn', 'sde', 'stl1', 'stl2', 'stl3', 'stl4', 'tsn', 'swvl1', 'swvl2', 'swvl3', 'swvl4']

for year in range(2020, 2021):
    # 获取某一年的文件
    flist = glob(f'/content/drive/MyDrive/2024/ai_pepper/era5/unzip/{year}/*')
    flist.sort()

    # 遍历该年的所有netcdf文件， 确定是part1还是part2
    for ncf in flist:
        ncfname = os.path.basename(ncf)
        yearmonth = ncfname.split('_')[0]
        month_int = int(yearmonth.split('-')[1])

        # 跳过已经完成的部分
        if year == 2020:
            if month_int not in [9, 10]:
                continue
        # 所有county数据保存在 2 个大表里
        # 每个大表都有一个 time 列表，然后每个 county 单独一个 dict
        # 每个 county 下面每个变量是一个表
        # all_dict1 = {'time': []}
        # all_dict2 = {'time': []}
        outd = {'time': []}
        # read basic variables
        with nc.Dataset(ncf,'r') as ds:
            var_all = list(ds.variables.keys())
            file_type = 0
            if ncf.split('_')[-1] == 'part1.nc':
                file_type = 1
            elif ncf.split('_')[-1] == 'part2.nc':
                file_type = 2
            else:
                raise ValueError('Unknown file type')
            if file_type == 1:
                for v in vlist1:
                    if v not in var_all:
                        raise ValueError(f'Part1 file. Variable not complete. Missing: {v}')
            elif file_type == 2:
                for v in vlist2:
                    if v not in var_all:
                        raise ValueError(f'Part2 file. Variable not complete. Missing: {v}')
            else:
                raise ValueError('Error during variable completeness check')

        # set variable list
        if file_type == 1:
            vlist = vlist1
            # continue
        elif file_type == 2:
            vlist = vlist2

        # 打开netcdf文件，先获取时间信息
        with nc.Dataset(ncf,'r') as ds:
            timevar = ds.variables['time'][:]
            timelist = [
                dt.datetime(1900, 1, 1, 0, 0) + dt.timedelta(hours = int(x)) for x in timevar
                ]

            # 根据 file_type 选择添加数据到 all_dict1 还是 all_dict2
            if file_type == 1:
                vlist = vlist1
            else:
                vlist = vlist2
            outd['time'].extend(timelist)
            # 遍历所有变量名，获取全美的数据
            for v in vlist:
                with open(f'/content/drive/MyDrive/2024/ai_pepper/era5/mask2_log_YuhuaSitu2.log','a') as fp:
                    now_str = dt.datetime.strftime(dt.datetime.now(), '%Y%m%dT%H%M%S')
                    fp.writelines(f'{ncf} - {v} - {now_str}\n')
                arr = ds.variables[v][:]   # [time, lat, lon]
                # arr2 = cp.asarray(arr1.data)
                # arr = cp.where(arr2 < -30000, cp.nan, arr2)

                for location in mask_dict.keys():
                    print(ncf)
                    print(v)
                    print(location)
                    if location not in outd.keys():
                        outd[location] = {}
                    if v not in outd[location].keys():
                        outd[location][v] = np.array([])
                    garr1 = mask_dict[location]
                    garr_inds = np.where(garr1 > -10)
                    # arr_masked = garr1 * arr
                    arr_masked = arr[:, garr_inds[0], garr_inds[1]]
                    arr_out = np.mean(arr_masked, axis=1)
                    # arr_out = cp.nanmean(arr_masked, axis=(1,2))
                    # print('einsum')
                    # arr_out = np.einsum("ijk,jk->ijk", arr, garr)
                    # arr_out = np.einsum("ijk->i", arr_masked)
                    print('saving to outd (dictionary)')
                    outd[location][v] = np.concatenate(
                        [outd[location][v], arr_out],
                        axis=0
                        )
                    clear_output(wait=True)


        # 每个月保存一次
        partname = ncfname.split('_')[1].split('.')[0]
        errlist = []
        for location in outd.keys():
            if location == 'time':
                continue
            os.makedirs(f'/content/drive/MyDrive/2024/ai_pepper/era5/county_env2/{location}', exist_ok=True)
            try:
                outd1 = {}
                for key in outd[location].keys():
                    outd1[key] = outd[location][key]# .get()
                t2 = pd.DataFrame(outd1)
                t2['datetime'] = outd['time']
                t2.to_csv(f'/content/drive/MyDrive/2024/ai_pepper/era5/county_env2/{location}/{location}_{yearmonth}_{partname}.csv', index=False)
            except:
                print(f'error with location: {location}')
                errlist.append(location)
